作业2

In [239]:
# 导入 pymc 模型包，和 arviz 等分析工具 
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import ipywidgets

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")

### 模型定义  
自变量：压力（stress），性别（sex）  
因变量：恋爱关系  
数据关系：  
$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1,\beta_2 & \stackrel{ind}{\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 +  \beta_1 X_{i1} + \beta_2 X_{i2}}}{1 + e^{\beta_0 +  \beta_1 X_{i1} + \beta_2 X_{i2}}} \\  
\text{priors:} & & \beta_0  &  \sim N\left(0, 0.5^2 \right)  \\  
							 & & \beta_1  & \sim N\left(0, 0.5^2 \right). \\  
               & & \beta_2  & \sim N\left(0, 0.5^2 \right). \\  
\end{array}  
$$

In [240]:
# 通过 pd.read_csv 加载数据 Data_Sum_HPP_Multi_Site_Share.csv
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')


# 选取METU站点的数据
df = df_raw[df_raw["Site"] == "METU"]

# 选取本节课涉及的变量
df = df[["romantic", "sex", "stress"]]

#重新编码，编码后的数据：1 = "yes"; 2 = "no"
df["romantic"] =  np.where(df['romantic'] == 2, 0, 1)
#0 表示男性，1表示女性
df["sex"] =  np.where(df['sex'] == 1, 0, 1)
#设置索引
df["index"] = range(len(df))
df = df.set_index("index")


In [241]:
df

,romantic,sex,stress
index,,,
0,1,0,43
1,0,1,45
2,1,1,40
3,0,1,48
4,1,1,36
...,...,...,...
176,0,1,47
177,1,1,38
178,1,1,43


In [242]:
# 因变量分布
plt.hist(df["romantic"])
plt.xticks([0,1])
plt.xlabel("romantic")
plt.ylabel("count")
sns.despine()
plt.show()

<Figure size 640x480 with 1 Axes>

In [243]:
# 绘制散点图
sns.scatterplot(data=df,
                x="stress",
                y="romantic",
                alpha=0.6)
# 设置x轴标题
plt.xlabel("stress")
# 设置y轴刻度
plt.yticks([0,1],['no','yes'])
sns.despine()

<Figure size 640x480 with 1 Axes>

In [244]:
with pm.Model() as log_model_ss:
    log_model_ss.add_coord('obs_id',df.index, mutable=True)
    sex = pm.MutableData("sex",df.sex, dims="obs_id") 
    stress  = pm.MutableData("stress",df.stress, dims="obs_id")                 
    
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')    


    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)                
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)     
    #sigma = pm.Exponential("sigma", 0.6)                 


    mu = pm.Deterministic("mu", beta_0 + beta_1*stress + beta_2*sex , dims="obs_id")

    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")       

    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

In [245]:
# 可视化模型（贝叶斯变量因果图）
# 我们使用 pymc 自带的 `model_to_graphviz` 方法来可视化模型中各变量的因果关系。
pm.model_to_graphviz(log_model_ss)

# MCMC采样 & 模型诊断

In [246]:
with log_model_ss:
    # MCMC 近似后验分布
    log_model_ss_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 53 seconds.


In [247]:
az.plot_trace(log_model_ss_trace,
              var_names=["beta_0","beta_1","beta_2"],
              figsize=(15,8),
              compact=False)
plt.show()

<Figure size 1500x800 with 6 Axes>

## 后验参数解释  

以下的结果显示：  
- $\beta_0 = 0.715$，那么 $e^{\beta_0} = 0.63$， 表明 X1 和 X2 为 0时，个体恋爱的可能性为 2.04。  
- $\beta_1 = -0.017$， $e^{\beta_1} = 0.98$， 表明压力分数每增加1个单位，个体恋爱的发生比变为之前的0.98倍。  $\beta_2 = 0.316$， $e^{\beta_2} = 1.37$， 女性的恋爱的发生为男性的1.37倍。  
-  $\beta_2$表示女性相对于男性的恋爱情况的差异.  
- 然而，$\beta_1$ 和$\beta_1$ 的94%HDI均包括0，说明压力分数和性别不能有效预测恋爱发生的概率。 

In [248]:
az.summary(log_model_ss_trace, var_names=["beta_0","beta_1","beta_2"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,0.715,0.441,-0.102,1.546,0.005,0.004,7062.0,8786.0,1.0
beta_1,-0.017,0.011,-0.039,0.004,0.000,0.000,7001.0,8820.0,1.0
beta_2,0.316,0.265,-0.201,0.793,0.003,0.002,10277.0,9348.0,1.0


In [249]:
# 通过 np.exp 将 beta 参数进行转换
az.plot_posterior(log_model_ss_trace, var_names=["beta_0","beta_1","beta_2"], transform = np.exp)
plt.show()

<Figure size 2208x552 with 3 Axes>

In [250]:
log_model_ss_trace

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data

In [251]:
#画出每个自变量对应的恋爱概率94%hdi值
az.plot_hdi(
    df.stress,
    log_model_ss_trace.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C1"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean = log_model_ss_trace.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = df.stress, 
             y= post_mean, 
             label="posterior mean", 
             color="C1")
#绘制真实数据散点图
sns.scatterplot(x = df.stress, 
                y= df.romantic,label="observed data", 
                color='#C00000', 
                alpha=0.5)
#设置图例位置
plt.legend(loc="upper right",
           bbox_to_anchor=(1.5, 1),
           fontsize=12)
sns.despine()

<Figure size 640x480 with 1 Axes>

In [252]:
#画出每个自变量对应的恋爱概率94%hdi值
az.plot_hdi(
    df.sex,
    log_model_ss_trace.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C1"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean = log_model_ss_trace.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = df.sex, 
             y= post_mean, 
             label="posterior mean", 
             color="C1")
#绘制真实数据散点图
sns.scatterplot(x = df.sex, 
                y= df.romantic,label="observed data", 
                color='#C00000', 
                alpha=0.5)
#设置图例位置
plt.legend(loc="upper right",
           bbox_to_anchor=(1.5, 1),
           fontsize=12)
sns.despine()

<Figure size 640x480 with 1 Axes>

# 新站点预测及评估

In [253]:
df = df_raw[df_raw["Site"] == "UCSB"]
df=df[["stress", "sex", "romantic"]]
df["romantic"]=np.where(df['romantic']==2,0,1)

df["sex"] =  np.where(df['sex'] == 1, 0, 1)
df["index"]=range(len(df))
df=df.set_index("index")
pred_coords ={"obs_id":df.index} 

with log_model_ss:
    pm.set_data({"stress":df.stress.values,
                 "sex": df.sex.values,
                 "y": df.romantic.values},
                 coords=pred_coords
                ) 

    prediction = pm.sample_posterior_predictive(log_model_ss_trace, 
                                                var_names=["y_est"],
                                                predictions=True,
                                                random_seed=84735)
print(df)
print(df['romantic'].isnull().sum(axis=0))

Sampling: [y_est]


       stress  sex  romantic
index                       
0          39    1         0
1          34    0         1
2          35    1         0
3          47    1         0
4          53    0         1
...       ...  ...       ...
103        48    1         1
104        45    1         1
105        27    0         1
106        46    1         1
107        30    1         1

[108 rows x 3 columns]
0


In [254]:
prediction.predictions_constant_data.y

<xarray.DataArray 'y' (obs_id: 108)>
array([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1],
      dtype=int32)
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 6 7 ... 100 101 102 103 104 105 106 107

In [255]:
prediction

Inference data with groups:
	> predictions
	> predictions_constant_data

In [256]:
# 提取储存在 predicitons中的预测值
y_pred = prediction.predictions["y_est"].stack(sample=("chain","draw","obs_id")).values
# 统计其中0和1的个数，并除以总数，得到0和1对应的比例值
y_pred_freq = np.bincount(y_pred)/len(y_pred)
#绘制柱状图
bars = plt.bar([0, 1], y_pred_freq, color="#70AD47")
#用于在柱状图上标明比例值
for bar, freq in zip(bars, y_pred_freq):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{freq:.2f}", ha='center', va='bottom')
#对刻度、标题、坐标轴标题进行设置
plt.xticks([0, 1])
plt.suptitle("Out-of-sample prediction(X=1)")
plt.xlabel("romantic")
plt.ylabel("proportion")
sns.despine()

<Figure size 640x480 with 1 Axes>

In [257]:
#stack(sample = ("chain", "draw")：将每一个X对应的4*5000个后验预测值合并到一个维度sample
#对于每一个X，需要计算其20000个值的平均值，因此将dim设置为sample
pred_pi = prediction.predictions.y_est.stack(sample = ("chain", "draw")).mean(dim="sample")
# 转换为数据框
pred_pi = pred_pi.to_dataframe()

#将原数据中的X 和Y存入数据框
pred_pi["stress"] = prediction.predictions_constant_data.stress.values
pred_pi["sex"] = prediction.predictions_constant_data.sex.values
pred_pi["romantic"] = prediction.predictions_constant_data.y.values
#根据分类标准（50-50）生成最终的分类结果
pred_pi["romantic_pred"] = np.where(pred_pi["y_est"] >= 0.5, 1, 0)
pred_pi

,y_est,stress,sex,romantic,romantic_pred
obs_id,,,,,
0,0.58780,39,1,0,1
1,0.53530,34,0,1,1
2,0.60710,35,1,0,1
3,0.55775,47,1,0,1
4,0.45385,53,0,1,0
...,...,...,...,...,...
103,0.55345,48,1,1,1
104,0.56330,45,1,1,1
105,0.55920,27,0,1,1


In [258]:
# 使用`pd.crosstab`生成混淆矩阵，前两个值表明你需要统计的列名
# 由于要生成一个2*2的联表，需要指定行的名称和列的名称
confusion_matrix = pd.crosstab(pred_pi["romantic"], pred_pi["romantic_pred"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,9,48
1,5,46


In [259]:
# 计算a b c d的数量
true_positive = 46
false_positive = 48
true_negative = confusion_matrix.at[0, 0]
false_negative = confusion_matrix.at[1, 0]
# 代入公式
accuracy = (true_positive + false_negative) /(true_positive + false_positive + true_negative + false_negative)
sensitivity = (true_positive) /(true_positive + false_negative)
specificity = (true_negative) / (true_negative + false_positive)

print("准确性:", accuracy)
print("敏感性:", sensitivity)
print("特异性:", specificity)

准确性: 0.4722222222222222
敏感性: 0.9019607843137255
特异性: 0.15789473684210525


# 后验评估

In [260]:
coords = {"obs_id": df.index}

with pm.Model(coords=coords) as log_model_ss:
    stress = pm.MutableData("stress", df.stress, dims="obs_id")
    sex = pm.MutableData("sex",df.sex, dims="obs_id") 

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5) 
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * stress + beta_2*sex , dims="obs_id")
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=df.romantic,dims="obs_id")

    log_model_ss_trace = pm.sample(draws=5000,                 
                                tune=1000,                  
                                chains=4,                     
                                discard_tuned_samples= True,
                            
                                random_seed=84735)
    log_model_ss_ppc = pm.sample_posterior_predictive(log_model_ss_trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 50 seconds.
Sampling: [y_est]


In [261]:
az.plot_ppc(log_model_ss_ppc, num_pp_samples=50)

<Axes: xlabel='y_est / y_est'>

<Figure size 640x480 with 1 Axes>

In [262]:
log_model_ss_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw"))

<xarray.DataArray 'y_est' (obs_id: 108, sample: 20000)>
array([[1, 0, 0, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 1],
       [1, 0, 1, ..., 1, 0, 1]])
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 6 7 ... 100 101 102 103 104 105 106 107
  * sample   (sample) object MultiIndex
  * chain    (sample) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3 3
  * draw     (sample) int64 0 1 2 3 4 5 6 ... 4993 4994 4995 4996 4997 4998 4999

In [263]:
#stack(sample = ("chain", "draw")：将每一个X对应的4*5000个后验预测值合并到一个维度sample
#对于每一个X，需要计算其20000个值的平均值，因此将dim设置为sample
pred_pi = log_model_ss_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw")).mean(dim="sample")
# 转换为数据框
pred_pi = pred_pi.to_dataframe()

In [264]:
#将原数据中的X 和Y存入数据框
pred_pi["stress"] = log_model_ss_ppc.constant_data.stress.values
pred_pi["sex"] = log_model_ss_ppc.constant_data.sex.values
pred_pi["romantic"] = log_model_ss_ppc.observed_data.y_est.values

#根据分类标准（50-50）生成最终的分类结果
pred_pi["romantic_pred"] = np.where(pred_pi["y_est"] >= 0.5, 1, 0)
pred_pi

,y_est,stress,sex,romantic,romantic_pred
obs_id,,,,,
0,0.46785,39,1,0,0
1,0.48020,34,0,1,0
2,0.48625,35,1,0,0
3,0.44635,47,1,0,0
4,0.43050,53,0,1,0
...,...,...,...,...,...
103,0.44545,48,1,1,0
104,0.45355,45,1,1,0
105,0.49755,27,0,1,0


In [265]:
# 使用`pd.crosstab`生成混淆矩阵，前两个值表明你需要统计的列名
# 由于要生成一个2*2的联表，需要指定行的名称和列的名称
confusion_matrix = pd.crosstab(pred_pi["romantic"], pred_pi["romantic_pred"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,55,2
1,50,1


In [266]:
# 计算a b c d的数量
true_positive = 0
false_positive = 0
true_negative = confusion_matrix.at[0, 0]
false_negative = confusion_matrix.at[1, 0]
# 代入公式
accuracy = (true_positive + false_negative) /(true_positive + false_positive + true_negative + false_negative)
sensitivity = (true_positive) /(true_positive + false_negative)
specificity = (true_negative) / (true_negative + false_positive)

print("准确性:", accuracy)
print("敏感性:", sensitivity)
print("特异性:", specificity)

准确性: 0.47619047619047616
敏感性: 0.0
特异性: 1.0


# 模型比较

In [271]:
df = df_raw[df_raw["Site"] == "METU"]
df=df[["stress", "avoidance","sex", "romantic"]]
df["romantic"]=np.where(df['romantic']==2,0,1)
df["index"]=range(len(df))
df["sex"] =  np.where(df['sex'] == 1, 0, 1)
df["index"] = range(len(df))
df=df.set_index("index")

In [270]:

with pm.Model() as log_model_ss:
    log_model_ss.add_coord('obs_id',df.index, mutable=True)
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)                
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)  
                     
   
    sex = pm.MutableData("sex",df.sex, dims="obs_id") 
    stress  = pm.MutableData("stress",df.stress, dims="obs_id")                 
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')    
     
    mu = pm.Deterministic("mu", beta_0 + beta_1*stress + beta_2*sex , dims="obs_id")

    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")       
     

    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")
    
    log_model_ss_trace = pm.sample(draws=5000,                  # 使用mcmc方法进行采样，draws为采样次数
                      tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                      chains=4,                     # 链数
                      discard_tuned_samples= True,
                      idata_kwargs={"log_likelihood": True},# tune的结果将在采样结束后被丢弃
                      random_seed=84735)
    
    log_model_ss_ppc = pm.sample_posterior_predictive(log_model_ss_trace)
    
    
log_model_ss_ppc

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 52 seconds.
Sampling: [y_est]


Inference data with groups:
	> posterior_predictive
	> observed_data
	> constant_data

In [233]:
with pm.Model() as log_model1:
   
    log_model1.add_coord('obs_id',df.index, mutable=True)
    avoidance = pm.MutableData("avoidance", df.avoidance, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * avoidance, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")
    
    log_model1_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735,
                                idata_kwargs={"log_likelihood": True})
    
    model1_ppc = pm.sample_posterior_predictive(log_model1_trace)

In [234]:
with pm.Model() as log_model2:
    log_model2.add_coord('obs_id',df.index, mutable=True)
    sex= pm.MutableData("sex", df.sex, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 *sex, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")
    log_model2_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735,
                                idata_kwargs={"log_likelihood": True})
    
    model2_ppc = pm.sample_posterior_predictive(log_model2_trace)

In [272]:
comparison_list = {
    "model1":log_model_ss_trace,
    "model(avoidance)": log_model1_trace,
    "model(sex)": log_model2_trace
}
az.compare(comparison_list)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model1,0,-123.212586,1.952272,0.000000,6.821121e-01,1.840120,0.000000,False,log
model(avoidance),1,-123.856725,1.559398,0.644139,3.178879e-01,2.386080,1.862057,False,log
model(sex),2,-125.059207,1.579195,1.846621,6.328271e-15,1.883228,0.917543,False,log
